In [2]:
import json
import pymysql as mc
import sqlalchemy
from sqlalchemy import create_engine
from getpass import getpass
import Credentials as cred
import pandas as pd
from pandas import pivot_table
import numpy as np

In [3]:
pw = getpass()

········


In [4]:
cnx = create_engine('mysql+pymysql://'+cred.user+':'+pw+'@'+cred.host+':3306/'+cred.db_name,echo=False)

In [5]:
review_df = pd.read_sql_query("SELECT business_id, date, review_id, stars, user_id FROM review",cnx)
review_df.head()

,business_id,date,review_id,stars,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,2011-02-25,x7mDIiDB3jEiPGPHOmDzyw,2,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,2012-11-13,dDl8zu1vWPdKGihJrwQbpw,5,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,2014-10-23,LZp4UX5zK3e-c5ZGSeo3kA,1,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,2011-02-25,Er4NBWCmCD4nM8_p1GRdow,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,2014-09-05,jsDu6QEJHbwP2Blom1PLCA,5,msQe1u7Z_XuqjGoqhB0J5g


In [6]:
view_df = pd.read_sql_query("SELECT * FROM tempView2",cnx)
view_df.head()

,business_id,categories
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho..."
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo..."
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw..."
3,6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai"
4,8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants"


In [7]:
df = review_df.merge(view_df,on='business_id')
df.head()

,business_id,date,review_id,stars,user_id,categories
0,iCQpiavjjPzJ5_3gPD5Ebg,2011-02-25,x7mDIiDB3jEiPGPHOmDzyw,2,msQe1u7Z_XuqjGoqhB0J5g,"Pizza, Restaurants"
1,iCQpiavjjPzJ5_3gPD5Ebg,2011-10-25,VluIpojME0yKOcRKI5L0Iw,3,D1_nrBr4dOrs7M82OaBRwQ,"Pizza, Restaurants"
2,iCQpiavjjPzJ5_3gPD5Ebg,2016-05-22,nsB9JAeghk0zOaSulSm9Yw,5,jEjc8HxeCeI22M4di0Y1aw,"Pizza, Restaurants"
3,iCQpiavjjPzJ5_3gPD5Ebg,2014-12-30,pcn01EMERpCpHEcoaohdEg,5,RTiaaHSN80VRTLSdNK5rfQ,"Pizza, Restaurants"
4,iCQpiavjjPzJ5_3gPD5Ebg,2013-03-24,0mFtAPTmInbXHqDjX9eiOg,5,hBmJs49zYcXdV4wo8F5iiA,"Pizza, Restaurants"


In [8]:
user_review_df = df.groupby(by='user_id').count()

In [16]:
frequent_posters = user_review_df[user_review_df['business_id']>50].index.values

In [17]:
condensed_df = df[df['user_id'].isin(frequent_posters)]

In [18]:
print (f'Original rows was {df.shape[0]}, limiting to just frequent posters was {condensed_df.shape[0]} rows')
print (f'Original distinct users was {df["user_id"].nunique()}, frequent posters make up {condensed_df["user_id"].nunique()}')
print (f'Original number restaurants was {df["business_id"].nunique()}, frequent posters make up {condensed_df["business_id"].nunique()} restaurants')

Original rows was 3654797, limiting to just frequent posters was 625109 rows
Original distinct users was 1031840, frequent posters make up 6348
Original number restaurants was 57173, frequent posters make up 49569 restaurants


In [22]:
temp = condensed_df.groupby(by=['user_id','business_id']).mean()
temp.head()
#piv_df = pivot_table(condensed_df,values='stars',index='user_id',
                     #columns='business_id',aggfunc=np.average)
#piv_df.head()

stars
user_id                business_id                  
---1lKK3aKOuomHnwAkAow --9e1ONYQuAa-CB_Rrw7Tw      4
                       1JgaRBX0oiRsvEhHF3ZMjw      1
                       2BbFeotL85cIaBjSq1SWiA      1
                       5aeR9KcboZmhDZlFscnYRA      5
                       5cbsjFtrntUAeUx51FaFTg      1

In [23]:
temp.shape[0]

625109